In [1]:
print("hello")

hello


In [8]:
# importing required liabraries
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [9]:
#Pinecone api key
PINECONE_API_KEY = "*****************************"

In [10]:
# function which loads the data
def data_loader(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)

    document = loader.load()

    return document

In [73]:
# Functions for creating chunks
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 20)
    chunks = text_splitter.split_documents(data)

    return chunks

In [ ]:
extracted_data = data_loader("data/")

In [74]:
chunks = text_split(extracted_data)[:1000] # ussing only 1000 chunks as pincone only allows 1000 vectors at a time

In [75]:
len(chunks)

1000

In [21]:
# function for downloading embedding model
def download_huggingface_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [22]:
# downloading model
embeddings = download_huggingface_embedding_model()

In [7]:
test = embeddings.embed_query("hello")
print(test)

[-0.06277173012495041, 0.0549587644636631, 0.052164845168590546, 0.08579001575708389, -0.0827489048242569, -0.07457298785448074, 0.06855474412441254, 0.018396401777863503, -0.08201131224632263, -0.03738486021757126, 0.01212488953024149, 0.003518301760777831, -0.004134277813136578, -0.043784454464912415, 0.021807288751006126, -0.005102697294205427, 0.019546590745449066, -0.04234873875975609, -0.11035966873168945, 0.005424531176686287, -0.05573474243283272, 0.028052419424057007, -0.023158712312579155, 0.028481345623731613, -0.05370963364839554, -0.052601564675569534, 0.033939216285943985, 0.04538865014910698, 0.0237184539437294, -0.0731208324432373, 0.054777760058641434, 0.017047306522727013, 0.08136036992073059, -0.00286271795630455, 0.011958097107708454, 0.07355853170156479, -0.09423743933439255, -0.0813620537519455, 0.040015410631895065, 0.0006922061438672245, -0.013393313623964787, -0.05453810095787048, 0.005151402670890093, -0.026139816269278526, 0.03680708631873131, -0.033959608525

In [18]:
# initializing pinecone
pc = Pinecone(api_key = PINECONE_API_KEY)
Index = pc.Index("chatbot")

In [115]:
    # Initialize an empty list to store the results
def Vectors()   : 
    Embed_data = []

        # Loop through the chunks and get the embeddings
    for index, chunk in enumerate(chunks):
    # Get the embedded vectors for the current chunk
        vectors = embeddings.embed_query(chunk.page_content)
            
        # Create a dictionary with the index and values
        result = {"id": str(index), "values": vectors, "metadata" : {"text" : chunk.page_content}}
            
        # Append the dictionary to the results list
        Embed_data.append(result)

        return Embed_data



In [116]:
print(Embed_data[0])

{'id': '0', 'values': [0.0017460356466472149, -0.03350284323096275, -0.03290391340851784, 0.0071680545806884766, -0.01460330467671156, 0.010261869058012962, -0.011515313759446144, 0.2293020784854889, -0.023232372477650642, 0.004120395053178072, -0.0365607887506485, 0.08592107892036438, 0.012972156517207623, 0.05221784859895706, -0.1023261621594429, -0.003139016218483448, -0.012687001377344131, 0.0004718389827758074, -0.02848588488996029, -0.050259191542863846, 0.01155097410082817, 0.0778064951300621, 0.0928281769156456, -0.013797277584671974, -0.016935111954808235, -0.025955883786082268, -0.0495651550590992, -0.046131301671266556, 0.007290540728718042, -0.013553300872445107, 0.03843944892287254, 0.06280478835105896, 0.018353784456849098, 0.008242836222052574, 0.0017155878012999892, -0.03986186906695366, -0.011638606898486614, 0.01644616387784481, 0.02559562399983406, 0.0910460576415062, 0.029672740027308464, -0.05416031926870346, -0.04576564580202103, -0.013853855431079865, 0.025773560

In [117]:
# making a json of the vectors
import json 
   
# Convert and write JSON object to file
with open("vector.json", "w") as outfile: 
    json.dump(Embed_data, outfile)

In [118]:
# vectors upsertion in index
Index.upsert(vectors=Embed_data)

{'upserted_count': 1000}

In [28]:
query = "headache"

In [29]:
result = Index.query(
    vector=embeddings.embed_query(query),
    top_k=5,
    include_metadata=True
)

In [30]:
for match in result.get("matches"):
    final_result = match.get("metadata")

In [36]:
prompt = PromptTemplate.from_template('''As a Good model generate a proper answer with good prensentation.
user will pass the question.

context : {context}
question : {question}

you have to read the data from the context and try to answer in a proper and professional way and try to format it as well. If you dont know simply says i dont know.
answer : ''')

In [37]:
llm = ChatOpenAI(openai_api_key="*****************************", model="gpt-3.5-turbo")

In [38]:
chain = LLMChain(llm=llm, prompt=prompt)

In [39]:
print(chain.run({"question" : query, "context" : final_result}))

For headaches, a common acupuncture point recommended for treatment is the "LI4" point, which is located in the web between the thumb and index finger on the back of the hand. To alleviate headaches, toothaches, sinus problems, and pain in the upper body, you can apply a pinching pressure to the LI4 point using the thumb and index finger of the other hand. This technique may help relieve discomfort and promote relaxation.
